In [1]:
# This plots the epochs at which RR Lyrae spectra were taken

# created 2017 Dec 19 by E.S.

In [1]:
import astropy
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from datetime import datetime
from astropy import time, coordinates as coord, units as u
from astropy.time import Time
from dateutil.parser import parse

In [2]:
stem = '/home/../../media/unasemaje/Seagate Expansion Drive/rrlyrae_data_reduction/unnorm_fits_files/'

In [3]:
import os
arr_files = os.listdir(stem) # list files

In [4]:
# read in each FITS file in the directory and obtain epoch from header

fileList_2012 = []
dateList_2012 = []
fileList_2013 = []
dateList_2013 = []
for f in range(0,len(arr_files)): # loop over filenames
    
    # retrieve header
    image, header = fits.getdata(stem+arr_files[f],
                                     0,
                                     header=True)
    
    # observation epoch
    epoch = header['DATE-OBS']+' '+header['UT']
    
    # parse
    epoch_dateTime = datetime.strptime(epoch, '%Y-%m-%d %H:%M:%S.%f')
    if (epoch_dateTime.year == 2012):
        fileList_2012.append(arr_files[f])
        dateList_2012.append(epoch_dateTime)
    else:
        fileList_2013.append(arr_files[f])
        dateList_2013.append(epoch_dateTime)

In [5]:
########################################
## MAKE PLOTS
########################################

In [6]:
# fcn for generating a plot to visualize epochs

def spec_epoch_plot(fileArray,epochArray,plotName):
    fig, ax = plt.subplots()
    fig.set_size_inches(200, 10)
    ax.scatter(epochArray, np.ones(len(epochArray)))
    
    #textPos = np.max(spectData.flux)+0.01
    [ax.text(epochArray[i], 2.5, fileArray[i], rotation='vertical') for i in range(len(epochArray))]
    
    ax.set_ylim([0,3.6])
    ax.set_xlim([np.min(epochArray),np.max(epochArray)])
    plt.savefig(plotName)

In [7]:
# write out plots of when spectra were observed

spec_epoch_plot(fileList_2012,dateList_2012,'test_2012.pdf')
spec_epoch_plot(fileList_2013,dateList_2013,'test_2013.pdf')

In [ ]:
########################################
## FIND ELAPSED BJDS
########################################

In [7]:
# set observatory coordinates 

loc_mcdonald = coord.EarthLocation.from_geodetic(lon=-104.0215753,lat=30.6715396,height=2076,ellipsoid='WGS84')
loc_macadam = coord.EarthLocation.from_geodetic(lon=-84.503712,lat=38.033891,height=298,ellipsoid='WGS84')

In [8]:
# convert UTC times to isot format, then compile into list of astropy Time object

t_spectra_2012_iso = [Time(dateList_2012[i].isoformat(), format='isot', scale='utc') for i in range(len(dateList_2012))]
t_spectra_2013_iso = [Time(dateList_2013[i].isoformat(), format='isot', scale='utc') for i in range(len(dateList_2013))]

In [9]:
# convert isot-format times to MJD or JD (Astropy seems to make more accurate conversion from MJD->BJD) 

t_spectra_2012_mjd = [t_spectra_2012_iso[i].mjd for i in range(len(t_spectra_2012_iso))]
t_spectra_2013_mjd = [t_spectra_2013_iso[i].mjd for i in range(len(t_spectra_2013_iso))]

t_spectra_2012_jd = [t_spectra_2012_iso[i].jd for i in range(len(t_spectra_2012_iso))]
t_spectra_2013_jd = [t_spectra_2013_iso[i].jd for i in range(len(t_spectra_2013_iso))]

In [10]:
# fcn to convert MJD to BJD times

def convert_mjd_to_bjd(mjdTimes,observatoryLoc,skyCoordObj):

    timesObj = time.Time(mjdTimes, format='mjd', scale='utc', location=observatoryLoc)
    ltt_bary = timesObj.light_travel_time(skyCoordObj)
    
    time_barycentre = timesObj.tdb + ltt_bary 
        
    # note the returned type is still mendaciously called 'mjd' 
    return np.add(time_barycentre.mjd,0.5) # I think 0.5 day is missing from the code

In [11]:
# fcn to convert MJD to HJD times

def convert_mjd_to_hjd(mjdTimes,observatoryLoc,skyCoordObj):

    timesObj = time.Time(mjdTimes, format='mjd', scale='utc', location=observatoryLoc)
    ltt_helio = timesObj.light_travel_time(skyCoordObj, 'heliocentric')
    
    times_heliocentre = timesObj.utc + ltt_helio 
        
    # note the returned type is still mendaciously called 'mjd' 
    return np.add(times_heliocentre.mjd,0.5) # I think 0.5 day is missing from the code

In [12]:
# combine all data for across-the-board comparison

allFileList = np.hstack((fileList_2012,fileList_2013))
allSpecEpochList_utc = np.hstack((dateList_2012,dateList_2013))
allSpecEpochList_mjd = np.hstack((t_spectra_2012_mjd,t_spectra_2013_mjd))
allSpecEpochList_jd = np.hstack((t_spectra_2012_jd,t_spectra_2013_jd))
#allSpecEpochList_bjd = np.hstack((t_spectra_2012_bjd,t_spectra_2013_bjd))

In [13]:
# read in star name, return file names and BJDs of spectra observations

def return_star_bjds(fileNames,mjdTimes,starNameFile,starNameGeneric,observatoryLoc):
    
    if len(fileNames) != len(mjdTimes): # something must be wrong!
        return
    
    # initialize a pandas dataframe
    df = pd.DataFrame()
    
    # make a star coordinate object
    coord_star = coord.SkyCoord.from_name(starNameGeneric)
    
    fileNamesThisStar = []
    mjdsThisStar = []
    hjdsThisStar = []
    bjdsThisStar = []
    for t in range(0,len(fileNames)):            
        if starNameFile in fileNames[t]:
            fileNamesThisStar.append(fileNames[t])
            mjdsThisStar.append(mjdTimes[t])
            hjdsThisStar.append(convert_mjd_to_hjd(mjdTimes[t],observatoryLoc,coord_star))
            bjdsThisStar.append(convert_mjd_to_bjd(mjdTimes[t],observatoryLoc,coord_star)) 
    
    #bjdsThisStar = np.add(bjdsThisStar,0.5)  # I think 0.5 day is missing from the code
    elapsed_bjd = np.subtract(bjdsThisStar,np.min(bjdsThisStar))
    
    df['filenames'] = fileNamesThisStar
    df['mjd'] = mjdsThisStar
    df['hjd'] = hjdsThisStar
    df['bjd'] = bjdsThisStar
    df['elapsed_bjd_since_spec_01'] = elapsed_bjd
    
    return df

In [14]:
# star names as they appear in the filenames
star_names_files = ['RW_Ari','X_Ari','UY_Cam','RR_Cet','SV_Eri',
              'VX_Her','RR_Leo','TT_Lyn','TV_Lyn','TW_Lyn',
              'RR_Lyr','V_535','V445','AV_Peg','BH_Peg',
              'AR_Per','RU_Psc','T_Sex','TU_UMa']

# star names for SIMBAD lookup
star_names_simbad = ['RW Ari','X Ari','UY Cam','RR Cet','SV Eri',
              'VX Her','RR Leo','TT Lyn','TV Lyn','TW Lyn',
              'RR Lyr','V535 Mon','V445 Oph','AV Peg','BH Peg',
              'AR Per','RU Psc','T Sex','TU UMa']

In [15]:
# find BJDs and concatenate everything into one dataframe

dfAll = pd.DataFrame()
for star in range(0,len(star_names_files)):
    df_thisStar = return_star_bjds(allFileList,allSpecEpochList_mjd,star_names_files[star],star_names_simbad[star],loc_mcdonald)
    dfAll = pd.concat([dfAll,df_thisStar])

In [16]:
# write out csv

dfAll.to_csv('junk.csv')

In [24]:
# show

dfAll

,filenames,mjd,hjd,bjd,elapsed_bjd_since_spec_01
0,RW_Ari_02.fits,56288.129929,56288.632936,56288.633699,0.039824
1,RW_Ari_01.fits,56288.090101,56288.593111,56288.593874,0.000000
2,RW_Ari_03.fits,56288.169575,56288.672579,56288.673341,0.079467
3,RW_Ari_04.fits,56289.084369,56289.587294,56289.588057,0.994182
4,RW_Ari_05.fits,56289.135552,56289.638472,56289.639235,1.045361
